# DNBLab Jupyter Notebook Tutorial

## OCR: Datenabfrage, Auslieferung und Textanalyse 

Entwurf: Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage zu digitalisierten Inhaltsverzeichnissen über die SRU-Schnittstelle. Das Tutorial umfasst eine exemplarische Abfrage, das temporäre Speichern der Inhaltsverzeichnisse als Textdateien und Durchsuchen der Volltexte nach einem beliebigen Stichwort. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden.

## Einrichten der Arbeitsumgebung <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die SRU-Schnittstelle wird BeautifulSoup https://www.crummy.com/software/BeautifulSoup/ und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit Pandas https://pandas.pydata.org/ können Elemente aus dem MARC21-Format ausgelesen werden.

In [1]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

## SRU-Abfrage mit Ausgabe in MARC21-xml<a class="anchor" id="Teil2"></a>

Die Funktion dnb_sru nimmt den Paramter "query" der SRU-Abfrage entgegen und liefert alle Ergebnisse als eine Liste von Records aus. Bei mehr als 100 Records werden weitere Datensätze mit "&startRecord=101" abgerufen (mögliche Werte 1 bis 99.000). Weitere Informationen und Funktionen der SRU- Schnittstelle werden unter https://www.dnb.de/sru beschrieben.

In [3]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

### Durchsuchen eines MARC-Feldes<a class="anchor" id="Teil3"></a>

Die Funktion parse_records nimmt als Parameter jeweils ein Record entgegen und sucht über xpath die gewünschte Informationen heraus und liefert diese als Dictionary zurück. Die Schlüssel-Werte-Paare können beliebig agepasst und erweitert werden. In diesem Fall werden nur die Permalinks zu den digitalisierten Inhaltsverzeichnissen als "link" ausgegeben.

In [4]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #link
    link = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        link = link[0].text
    except:
        link = "unknown"
        
    meta_dict = {"link":link + '/text'}
    
    return meta_dict

Über verschiedenen Indices https://services.dnb.de/sru/dnb?operation=explain&version=1.1 kann die SRU-Abfrage "dnb_sru" mittels CQL https://www.dnb.de/DE/Service/Hilfe/Katalog/kataloghilfeExpertensuche.html eingeschränkt werden. Im Folgenden Code wird die Abfrage über das Stichwort "Sandwespe" im Volltextindex der digitalisierten Inhaltsverzeichnisse eingeschränkt. Durch Anpassen der SRU-Abfrage kann die Trefferliste beliebig geändert werden.

In [5]:
records = dnb_sru('inh=Nähmaschine')
print(len(records), 'Ergebnisse')

750 Ergebnisse


## Beispielanzeige zur weiteren Bearbeitung <a class="anchor" id="Teil4"></a>

Mit der Bibliothek Pandas für Python wird das Ergebnis (Dictionary-Element "link") als Dataframe ausgegeben.

In [6]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

,link
0,http://deposit.dnb.de/cgi-bin/dokserv?id=043b0...
1,http://deposit.dnb.de/cgi-bin/dokserv?id=f6d16...
2,http://deposit.dnb.de/cgi-bin/dokserv?id=1b6e2...
3,http://deposit.dnb.de/cgi-bin/dokserv?id=1f524...
4,http://deposit.dnb.de/cgi-bin/dokserv?id=f4e7e...
...,...
745,https://d-nb.info/363621814/04/text
746,https://d-nb.info/361208944/04/text
747,https://d-nb.info/362024774/04/text
748,https://d-nb.info/580171256/04/text


Die Ausgabe der ermittelten Links kann je nach Bedarf über verschiedene Funktionen erfolgen:

In [7]:
#print(df.to_string(index=False))
#HTML(df.to_html(index=False))
#document = df.to_dict(orient='list')
#print(document)

Mit der folgenden Funktion df.to_csv() werden die Ergebnisse als "links.csv" in das Jupyter-Verzeichnins der Einstiegsseite abgelegt und können dort heruntergeladen werden. 

In [8]:
df.to_csv("links.csv", index=False)

Mit wget werden alle in der CSV-Datei gespeicherten Links heruntergeladen und als Textdateien (text, text.1, text.2, usw.) im temporären Jupyter-Verzeichnis gespeichert. 

In [9]:
!wget -i links.csv

--2021-08-05 08:31:49--  http://link/
Resolving link (link)... failed: No address associated with hostname.
wget: unable to resolve host address ‘link’
--2021-08-05 08:31:49--  http://deposit.dnb.de/cgi-bin/dokserv?id=043b027d59f04e619522dde80bd12406&prov=M&dok_var=1&dok_ext=htm/text
Resolving deposit.dnb.de (deposit.dnb.de)... 193.175.100.44
Connecting to deposit.dnb.de (deposit.dnb.de)|193.175.100.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘dokserv?id=043b027d59f04e619522dde80bd12406&prov=M&dok_var=1&dok_ext=htm%2Ftext’

dokserv?id=043b027d     [ <=>                ]     910  --.-KB/s    in 0s      

2021-08-05 08:31:49 (29.7 MB/s) - ‘dokserv?id=043b027d59f04e619522dde80bd12406&prov=M&dok_var=1&dok_ext=htm%2Ftext’ saved [910]

--2021-08-05 08:31:49--  http://deposit.dnb.de/cgi-bin/dokserv?id=f6d160d66f69408bb6fcae6a33a8d0a6&prov=M&dok_var=1&dok_ext=htm/text
Reusing existing connection to deposit.dnb.de:80.
HTTP reque

HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘dokserv?id=61b3cc20f48e497ba3de418257398b6a&prov=M&dok_var=1&dok_ext=htm%2Ftext’

dokserv?id=61b3cc20     [ <=>                ]   1.43K  --.-KB/s    in 0s      

2021-08-05 08:31:52 (64.1 MB/s) - ‘dokserv?id=61b3cc20f48e497ba3de418257398b6a&prov=M&dok_var=1&dok_ext=htm%2Ftext’ saved [1462]

--2021-08-05 08:31:52--  http://deposit.dnb.de/cgi-bin/dokserv?id=639f5717b678437d8c5e793a41359e62&prov=M&dok_var=1&dok_ext=htm/text
Reusing existing connection to deposit.dnb.de:80.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘dokserv?id=639f5717b678437d8c5e793a41359e62&prov=M&dok_var=1&dok_ext=htm%2Ftext’

dokserv?id=639f5717     [ <=>                ]   1.01K  --.-KB/s    in 0s      

2021-08-05 08:31:52 (47.6 MB/s) - ‘dokserv?id=639f5717b678437d8c5e793a41359e62&prov=M&dok_var=1&dok_ext=htm%2Ftext’ saved [1033]

--2021-08-05 08:31:52--  http://deposit.dnb.de/cg

HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘dokserv?id=b4196eb358ee4882867554b81f9947ea&prov=M&dok_var=1&dok_ext=htm%2Ftext’

dokserv?id=b4196eb3     [ <=>                ]   1.06K  --.-KB/s    in 0s      

2021-08-05 08:31:56 (48.2 MB/s) - ‘dokserv?id=b4196eb358ee4882867554b81f9947ea&prov=M&dok_var=1&dok_ext=htm%2Ftext’ saved [1081]

--2021-08-05 08:31:56--  https://d-nb.info/1197007741/04/text
Connecting to d-nb.info (d-nb.info)|193.175.100.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4769 (4.7K) [text/plain]
Saving to: ‘text.11’

text.11             100%[===================>]   4.66K  --.-KB/s    in 0s      

2021-08-05 08:31:57 (103 MB/s) - ‘text.11’ saved [4769/4769]

--2021-08-05 08:31:57--  http://deposit.dnb.de/cgi-bin/dokserv?id=1e77ad0fff6a4d57bcf01d272c3d45da&prov=M&dok_var=1&dok_ext=htm/text
Connecting to deposit.dnb.de (deposit.dnb.de)|193.175.100.44|:80... connected.
HTTP request sent, awaitin

HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘dokserv?id=3f8addb899cb490eb6163c703f427d5a&prov=M&dok_var=1&dok_ext=htm%2Ftext’

dokserv?id=3f8addb8     [ <=>                ]     782  --.-KB/s    in 0s      

2021-08-05 08:32:01 (35.6 MB/s) - ‘dokserv?id=3f8addb899cb490eb6163c703f427d5a&prov=M&dok_var=1&dok_ext=htm%2Ftext’ saved [782]

--2021-08-05 08:32:01--  https://d-nb.info/1170088635/04/text
Connecting to d-nb.info (d-nb.info)|193.175.100.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 757 [text/plain]
Saving to: ‘text.23’

text.23             100%[===================>]     757  --.-KB/s    in 0s      

2021-08-05 08:32:02 (54.4 MB/s) - ‘text.23’ saved [757/757]

--2021-08-05 08:32:02--  https://d-nb.info/1181981832/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 1738 (1.7K) [text/plain]
Saving to: ‘text.24’

text.24             100%[=============

dokserv?id=f4b7cf73     [ <=>                ]   1.27K  --.-KB/s    in 0s      

2021-08-05 08:32:08 (55.7 MB/s) - ‘dokserv?id=f4b7cf732da745628af4a99454297bab&prov=M&dok_var=1&dok_ext=htm%2Ftext’ saved [1302]

--2021-08-05 08:32:08--  http://deposit.dnb.de/cgi-bin/dokserv?id=1144d10332c04e8cb94076c11ea84dab&prov=M&dok_var=1&dok_ext=htm/text
Reusing existing connection to deposit.dnb.de:80.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘dokserv?id=1144d10332c04e8cb94076c11ea84dab&prov=M&dok_var=1&dok_ext=htm%2Ftext’

dokserv?id=1144d103     [ <=>                ]     961  --.-KB/s    in 0s      

2021-08-05 08:32:08 (44.1 MB/s) - ‘dokserv?id=1144d10332c04e8cb94076c11ea84dab&prov=M&dok_var=1&dok_ext=htm%2Ftext’ saved [961]

--2021-08-05 08:32:08--  http://deposit.dnb.de/cgi-bin/dokserv?id=56ff06c728b446a58106361a1e267165&prov=M&dok_var=1&dok_ext=htm/text
Reusing existing connection to deposit.dnb.de:80.
HTTP request sent, awaiting response... 

...

HTTP request sent, awaiting response... 200 OK
Length: 5674 (5.5K) [text/plain]
Saving to: ‘text.476’

text.476            100%[===================>]   5.54K  --.-KB/s    in 0s      

2021-08-05 08:34:23 (207 MB/s) - ‘text.476’ saved [5674/5674]

--2021-08-05 08:34:23--  https://d-nb.info/457572604/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 518 [text/plain]
Saving to: ‘text.477’

text.477            100%[===================>]     518  --.-KB/s    in 0s      

2021-08-05 08:34:23 (43.4 MB/s) - ‘text.477’ saved [518/518]

--2021-08-05 08:34:23--  https://d-nb.info/457572590/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 503 [text/plain]
Saving to: ‘text.478’

text.478            100%[===================>]     503  --.-KB/s    in 0s      

2021-08-05 08:34:23 (40.2 MB/s) - ‘text.478’ saved [503/503]

--2021-08-05 08:34:23--  https://d-nb.info/457572574/04/text
Re

HTTP request sent, awaiting response... 200 OK
Length: 875 [text/plain]
Saving to: ‘text.500’

text.500            100%[===================>]     875  --.-KB/s    in 0s      

2021-08-05 08:34:29 (72.8 MB/s) - ‘text.500’ saved [875/875]

--2021-08-05 08:34:29--  https://d-nb.info/58009393X/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 9632 (9.4K) [text/plain]
Saving to: ‘text.501’

text.501            100%[===================>]   9.41K  --.-KB/s    in 0s      

2021-08-05 08:34:30 (68.6 MB/s) - ‘text.501’ saved [9632/9632]

--2021-08-05 08:34:30--  https://d-nb.info/365935980/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 6263 (6.1K) [text/plain]
Saving to: ‘text.502’

text.502            100%[===================>]   6.12K  --.-KB/s    in 0s      

2021-08-05 08:34:30 (137 MB/s) - ‘text.502’ saved [6263/6263]

--2021-08-05 08:34:30--  https://d-nb.info/368227960/

HTTP request sent, awaiting response... 200 OK
Length: 5397 (5.3K) [text/plain]
Saving to: ‘text.546’

text.546            100%[===================>]   5.27K  --.-KB/s    in 0s      

2021-08-05 08:34:42 (70.6 MB/s) - ‘text.546’ saved [5397/5397]

--2021-08-05 08:34:42--  https://d-nb.info/366648837/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 15717 (15K) [text/plain]
Saving to: ‘text.547’

text.547            100%[===================>]  15.35K  --.-KB/s    in 0.02s   

2021-08-05 08:34:42 (967 KB/s) - ‘text.547’ saved [15717/15717]

--2021-08-05 08:34:42--  https://d-nb.info/361270046/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 837 [text/plain]
Saving to: ‘text.548’

text.548            100%[===================>]     837  --.-KB/s    in 0s      

2021-08-05 08:34:43 (65.8 MB/s) - ‘text.548’ saved [837/837]

--2021-08-05 08:34:43--  https://d-nb.info/57220884

HTTP request sent, awaiting response... 200 OK
Length: 4428 (4.3K) [text/plain]
Saving to: ‘text.592’

text.592            100%[===================>]   4.32K  --.-KB/s    in 0s      

2021-08-05 08:34:55 (102 MB/s) - ‘text.592’ saved [4428/4428]

--2021-08-05 08:34:55--  https://d-nb.info/366708457/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 8982 (8.8K) [text/plain]
Saving to: ‘text.593’

text.593            100%[===================>]   8.77K  --.-KB/s    in 0s      

2021-08-05 08:34:56 (71.1 MB/s) - ‘text.593’ saved [8982/8982]

--2021-08-05 08:34:56--  https://d-nb.info/579200442/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 1201 (1.2K) [text/plain]
Saving to: ‘text.594’

text.594            100%[===================>]   1.17K  --.-KB/s    in 0s      

2021-08-05 08:34:56 (92.5 MB/s) - ‘text.594’ saved [1201/1201]

--2021-08-05 08:34:56--  https://d-nb.info/

HTTP request sent, awaiting response... 200 OK
Length: 1694 (1.7K) [text/plain]
Saving to: ‘text.638’

text.638            100%[===================>]   1.65K  --.-KB/s    in 0s      

2021-08-05 08:35:09 (137 MB/s) - ‘text.638’ saved [1694/1694]

--2021-08-05 08:35:09--  https://d-nb.info/361548354/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 2730 (2.7K) [text/plain]
Saving to: ‘text.639’

text.639            100%[===================>]   2.67K  --.-KB/s    in 0s      

2021-08-05 08:35:10 (66.5 MB/s) - ‘text.639’ saved [2730/2730]

--2021-08-05 08:35:10--  https://d-nb.info/366134248/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 13396 (13K) [text/plain]
Saving to: ‘text.640’

text.640            100%[===================>]  13.08K  --.-KB/s    in 0.02s   

2021-08-05 08:35:10 (842 KB/s) - ‘text.640’ saved [13396/13396]

--2021-08-05 08:35:10--  https://d-nb.info

Die heruntergeladenen Textdateien können jetzt nach einem Suchwort, z.B. search = "Wild" durchsucht werden. Hierbei wird die Groß- und Kleinschreibung beachtet.
Die Treffer werden mit Angabe der Zeile und Datei ausgegeben. Dabei entspricht die Dateibenennung den im Verzeichnis heruntergeladen Textdateien (text, text1, text2 usw.).
Das Suchwort kann beliebig geändert und durch Ausführen des Codes die Suche angepasst werden.

In [ ]:
search = 'Handarbeit'

filename = 'text'
with open(filename) as f:
    for num, line in enumerate(f, 1):
        if search in line:
            print('%s - found at line in text:' % search, num)
        else:
            print('Kein Ergebnis vorhanden')
            
filename2 = 'text.1'
with open(filename2) as f:
    for num, line in enumerate(f, 1):
        if search in line:
            print('%s - found at line in text.1:' % search, num)
        else:
            print('Kein Ergebnis vorhanden')

In [ ]:
#Import os module
import os

# Ask the user to enter string to search
search_path = input("Enter directory path to search : ")
file_type = input("File Type : ")
search_str = input("Enter the search string : ")

# Append a directory separator if not already present
if not (search_path.endswith("/") or search_path.endswith("\\") ): 
        search_path = search_path + "/"
                                                          
# If path does not exist, set search path to current directory
if not os.path.exists(search_path):
        search_path ="."

# Repeat for each file in the directory  
for fname in os.listdir(path=search_path):

   # Apply file type filter   
   if fname.endswith(file_type):

        # Open file for reading
        fo = open(search_path + fname)

        # Read the first line from the file
        line = fo.readline()

        # Initialize counter for line number
        line_no = 1

        # Loop until EOF
        while line != '' :
                # Search for string in line
                index = line.find(search_str)
                if ( index != -1) :
                    print(fname, "[", line_no, ",", index, "] ", line, sep="")

                # Read next line
                line = fo.readline()  

                # Increment line counter
                line_no += 1
        # Close the files
        fo.close()

## Analysieren von OCR-Dateien

## OCR-Datei nach Suchbegriffen durchsuchen

Zum Analysieren von OCR-Dateeien wird die Library "re" benötigt. 

In [1]:
import re

In diesem Tutorial nutzen wir eine OCR-Datei des DNBLab. Diese wird im (Binder)Verzeichnis abgelegt. Damit die Datei durchsuchbar wird, wird der Text des Dokuments in einen String umgewandelt. Für eine bessere Durchsuchbarkeit, werden alle Zeichen des Dokuments in Kleinbuchstaben umgewandelt. 

In [ ]:
# Öffnen der Datei, 'r' regelt den Rechtezugriff
document_text = open ('102655246X_OCR.txt', 'r') 

# Umwandeln der Datei in einen String
text_string = document_text.read()

# Umwandeln der Textdatei in Kleinbuchstaben
text_string1 = text_string.lower()

Für die weitere Verarbeitung wird eine Liste erstelt, die die Inhalte der OCR-Datei beinhaltet.

In [ ]:
lines = [] #Erstellt eine leere Liste
for line in text_string1.splitlines():
    lines.append(line) #befüllt die Liste mit der jeweiligen Zeile

Suchen wir nun einen bestimmten Begriff, so nutzen wir einen re.search-Befehl, um die Position des Wortes im Dokument zu finden. 

In [ ]:
for element in lines:
    a = re.search('file', element)
    if a: 
        print(a)

Alternativ kann die Search-Funktion, die bekannt aus der Analyse des Inhaltsverzeichnissen ist, genutzt werden. 

In [ ]:
search = 'krieg'

filename = '102655246X_OCR.txt'
with open(filename) as f:
    for num, line in enumerate(f, 1):
        if search in line:
            print('%s - found at line in text:' % search, num)

## Häufigkeitsanalyse von regulären Ausdrucken 

Nach dem Import der Libary re und dem Öffnen OCR-Datei sowie der Umwandlung in String und Kleinschreibung, muss im Code ein regulärer Ausdruck definiert werden. 

In [31]:
import re 

# Öffnen der Datei, 'r' regelt den Rechtezugriff
document_text = open ('102655246X_OCR.txt', 'r') 

# Umwandeln der Datei in einen String
text_string = document_text.read()

# Umwandeln der Textdatei in Kleinbuchstaben
text_string1 = text_string.lower()

## Anmerkung, hier muss noch der Teil zu regulären Ausdrücken + Erklärungen der einzelnen Abschnitte hin 
for r in lines:
    
    a = re.match('document_text', text_string)
    
print(a) 



None


## Ausgabe der OCR 

Zur Weiterverarbeitung ist es auch möglich, sich den Inhalt der OCR-Datei direkt im Dokument ausgeben zu lassen. Dafür genügt es den Inhalt der OCR-Datei als Liste zu definieren und sich über einen Print-Befehl ausgeben zu lassen. 

In [ ]:
##OCR-Ausgabe :D - Testen! 

document_text = open ('102655246X_OCR.txt', 'r') 
text_string = document_text.read ()
text_string = document_text.read (). lower ()

with open("102655246X_OCR.txt", "r") as tf:
    lines = tf.read().split('\n')

for r in lines:
    
    a = re.match('document_text', text_string)
    
print(lines) 



## Codeschnippsel für reguläre Ausdrücke beginnen hier 

In [16]:
match_pattern = re.findall ('102655246X_OCR.txt' '\ b [a-z] 3,15  \ b ') 



TypeError: findall() missing 1 required positional argument: 'string'

In [ ]:
match_pattern: count = frequency.get (word, 0) Frequenz [word] = count + 1

In [ ]:
frequency_list = frequency.keys ()

In [ ]:
frequency_list: Wörter drucken, Häufigkeit [Wörter]

SyntaxError: invalid syntax (<ipython-input-1-df4597f046d6>, line 1)

In [ ]:
Ideen:
    Eine Suchanfrage, die alle regulären Ausdrücke findet
    Eine Suchanfrage für einen bestimmen Begriff 